In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from utils import GraphUtils,ExpUtils
import pandas as pd
import tensorflow as tf
from tensorflow_probability import edward2 as ed
from tensorflow.python import tf2
if not tf2.enabled():
    import tensorflow.compat.v2 as tf
    tf.enable_v2_behavior()
    assert tf2.enabled()
from tqdm import tqdm
import tensorflow_probability as tfp
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve,roc_auc_score, classification_report
from matplotlib import pyplot as plt

sns.reset_defaults()
sns.set_context(context='talk',font_scale=0.7)
plt.rcParams['image.cmap'] = 'viridis'

%matplotlib inline

tfd = tfp.distributions



## Load the structure of the Bayesian network

In [ ]:
network_structure = GraphUtils.load_graph(r"graph_structures/toy_example_structure.xlsx")
GraphUtils.visualise_network(network_structure)

## Load the generated data from the structure.
We have not yet open sourced our Bayesian systhetic tool which was used to generate the data here 

In [ ]:
toy_data = pd.read_csv('data/toy_data.csv')
toy_data.describe()

## Plot the distributions of the idependent variables

In [ ]:
#encoder per variable
x1_encoder = OneHotEncoder().fit(np.asarray(toy_data['x1']).reshape(-1,1))
x2_encoder = OneHotEncoder().fit(np.asarray(toy_data['x2']).reshape(-1,1))
x3_encoder = OneHotEncoder().fit(np.asarray(toy_data['x3']).reshape(-1,1))

m1_encoder = OneHotEncoder().fit(np.asarray(toy_data['m1']).reshape(-1,1))
m2_encoder = OneHotEncoder().fit(np.asarray(toy_data['m2']).reshape(-1,1))

y_encoder = OneHotEncoder().fit(np.asarray(toy_data['y']).reshape(-1,1))

In [ ]:
# Plot the distributions to be used in the paper:
plt.bar(['$x_{11}$','$x_{12}$'],[.3,.7],width=[0.3,.3],color='b')
plt.show()

plt.bar(['$x_{21}$','$x_{22}$','$x_{23}$'],[.1,.5,.4],width=[0.4,.4,.4],color='b')
plt.show()

plt.bar(['$x_{31}$','$x_{32}$','$x_{33}$'],[.3,.3,.3],width=[0.4,.4,.4],color='b')
plt.show()

plt.bar(['$m_{11}$','$m_{12}$','$m_{13}$'],[.5,.3,.2],width=[0.4,.4,.4],color='g')
plt.show()

plt.bar(['$m_{21}$','$m_{22}$'],[.6,.4],width=[0.3,.3],color='g')
plt.show()

plt.bar(['$y_{11}$','$y_{12}$'],[.5,.5],width=[0.3,.3],color='g')
plt.show()


## Define the probability distributions

In [ ]:
temperature = .7
x1_dist = tfd.RelaxedOneHotCategorical(temperature, probs=[.3,.7])
x2_dist = tfd.RelaxedOneHotCategorical(temperature, probs=[.1,.5,.4])
x3_dist = tfd.RelaxedOneHotCategorical(temperature, probs=[.3,.3,.3])
m1_prior = tfd.OneHotCategorical(probs=[.5,.3,.2]) 
m2_prior = tfd.OneHotCategorical(probs=[.6,.4]) 
y_prior = tfd.OneHotCategorical(probs=[.5,.5])

n = toy_data.shape[0]
x1_prior = tfd.OneHotCategorical(probs=np.asarray((toy_data['x1'].value_counts()/n)[x1_encoder.categories_[0]]))
x2_prior = tfd.OneHotCategorical(probs=np.asarray((toy_data['x2'].value_counts()/n)[x2_encoder.categories_[0]]))
x3_prior = tfd.OneHotCategorical(probs=np.asarray((toy_data['x3'].value_counts()/n)[x3_encoder.categories_[0]]))

## Build estimators for the conditional distributions simulating real classifiers

## build p(m1|x1,x2) 

In [ ]:
toy_data['m1_lbl'] =  LabelEncoder().fit_transform(toy_data['m1'])
x1_feature = x1_encoder.transform(np.asarray(toy_data['x1']).reshape(-1,1)).toarray()
x2_feature = x2_encoder.transform(np.asarray(toy_data['x2']).reshape(-1,1)).toarray()

feature_names_m1 = np.hstack((x1_encoder.categories_,x2_encoder.categories_))[0]
data_m1 = np.hstack((x1_feature,x2_feature))

feature_columns=[]
for feature_name in feature_names_m1:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_m1 = pd.DataFrame(columns = feature_names_m1,data=data_m1)
train_input_fn = ExpUtils.make_input_fn(data_m1, toy_data['m1_lbl'])
eval_input_fn = ExpUtils.make_input_fn(data_m1, toy_data['m1_lbl'], num_epochs=1, shuffle=False)

m1_x1_x2_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=3)
m1_x1_x2_est.train(train_input_fn)
result = m1_x1_x2_est.evaluate(eval_input_fn)

In [ ]:
# print results from TF
print(result)
print('*****************************')
pred_dicts = list(m1_x1_x2_est.predict(eval_input_fn))
preds = [int(pred['classes'][0]) for pred in pred_dicts]

print(classification_report(toy_data['m1_lbl'],preds))

## build p(m2|x2,x3) 

In [ ]:
# build p(m2|x2,x3) 
toy_data['m2_lbl'] =  LabelEncoder().fit_transform(toy_data['m2'])
x2_feature = x2_encoder.transform(np.asarray(toy_data['x2']).reshape(-1,1)).toarray()
x3_feature = x3_encoder.transform(np.asarray(toy_data['x3']).reshape(-1,1)).toarray()

feature_names_m2 = np.hstack((x2_encoder.categories_,x3_encoder.categories_))[0]
data_m2 = np.hstack((x2_feature,x3_feature))

feature_columns=[]
for feature_name in feature_names_m2:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_m2 = pd.DataFrame(columns = feature_names_m2,data=data_m2)
train_input_fn_m2 = ExpUtils.make_input_fn(data_m2, toy_data['m2_lbl'])
eval_input_fn_m2 = ExpUtils.make_input_fn(data_m2, toy_data['m2_lbl'], num_epochs=1, shuffle=False)

m2_x2_x3_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=2)
m2_x2_x3_est.train(train_input_fn_m2)
result = m2_x2_x3_est.evaluate(eval_input_fn_m2)

In [ ]:
# print results from TF
print(result)
print('*****************************')
pred_dicts = list(m2_x2_x3_est.predict(eval_input_fn_m2))
preds = [int(pred['classes'][0]) for pred in pred_dicts]

print(classification_report(toy_data['m2_lbl'],preds))

## build p(y|m1,m2) 

In [ ]:
toy_data['y_lbl'] =  LabelEncoder().fit_transform(toy_data['y'])
m1_feature = m1_encoder.transform(np.asarray(toy_data['m1']).reshape(-1,1)).toarray()
m2_feature = m2_encoder.transform(np.asarray(toy_data['m2']).reshape(-1,1)).toarray()

feature_names_y = np.hstack((m1_encoder.categories_,m2_encoder.categories_))[0]
data_y = np.hstack((m1_feature,m2_feature))

feature_columns=[]
for feature_name in feature_names_y:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_y = pd.DataFrame(columns = feature_names_y,data=data_y)
train_input_fn_y = ExpUtils.make_input_fn(data_y, toy_data['y_lbl'])
eval_input_fn_y = ExpUtils.make_input_fn(data_y, toy_data['y_lbl'], num_epochs=1, shuffle=False)

y_m1_m2_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=2)
y_m1_m2_est.train(train_input_fn_y)
result = y_m1_m2_est.evaluate(eval_input_fn_y)

In [ ]:
# print results from TF
print(result)
print('*****************************')
pred_dicts = list(y_m1_m2_est.predict(eval_input_fn_y))
preds = [int(pred['classes'][0]) for pred in pred_dicts]

print(classification_report(toy_data['y_lbl'],preds))

## Experiments

In [ ]:
# prepare the configuration per experiment
conf_df = network_structure.copy()

In [ ]:
'''add the confirguration for each of the variables in the graph:
    - estimator: if applicable, the conditional pdf estimated as a TF model
    - prior: the prior distribution of the variable
    - encoder: if applicable, the encoder assocaited with the variable
    - feature_name: if applicable, the names of the input features for the model associated with the model
'''
estimators_ = ['','','','m1_x1_x2_est','m2_x2_x3_est','y_m1_m2_est']
priors_ = ['x1_prior','x2_prior','x3_prior','m1_prior','m2_prior','y_prior']
encoders_ = ['x1_encoder','x2_encoder','x3_encoder','m1_encoder','m2_encoder','y_encoder']
feature_names_ = ['','','','feature_names_m1','feature_names_m2','feature_names_y']

In [ ]:
conf_df['estimator'] = estimators_
conf_df['prior'] = priors_
conf_df['encoder'] = encoders_
conf_df['feature_names'] = feature_names_

In [ ]:
conf_df

In [ ]:
# Implement Eq 1 in the paper
def joint_prob(df,conf_df, head):
    if head not in set(conf_df['node']):
        print('head is not a node')
        return
    # get dependencies
    current_node = conf_df[conf_df['node']==head]
    prior_str = current_node['prior'].iloc[0]
    if ''!=prior_str:
        prior_dist = eval(current_node['prior'].iloc[0])
    else:
        prior_dist = None
    dep_nodes = current_node.parent_node.str.split(',').tolist()[0]
    if len(dep_nodes)==1 and dep_nodes[0]=='':#no dependencies end of recurssion
        # encode the features and then calcualte the priors if they are defined
        if prior_dist is not None:
            #load the encoder and encode the input data
            encoder_ = eval(current_node['encoder'].iloc[0])
            encoded_features = encoder_.transform(np.asarray(df[head]).reshape(-1,1)).toarray()
            tmp = tf.convert_to_tensor(np.tile(prior_dist.prob(encoded_features),(encoded_features.shape[1],1)).T) 
            output = tf.convert_to_tensor(encoded_features,dtype=tf.float32)* tmp 
            return output
        return 1.0
    n_probs = None
    for n in dep_nodes:
        probs_ = joint_prob(df,conf_df, head=n)
        if n_probs is None:
            n_probs = probs_
        else:
            n_probs = tf.concat([n_probs,probs_],axis=1)
    est = eval(current_node['estimator'].iloc[0])
    feature_names = eval(current_node['feature_names'].iloc[0])
    eval_input_fn = ExpUtils.make_input_fn(pd.DataFrame(columns=feature_names,data=n_probs.numpy()),
                                               None,num_epochs=1, shuffle=False)
    cond = ExpUtils.conditional_prob(eval_input_fn,est)
    
    if prior_dist is not None:
        prior = prior_dist.prob(cond)
        return tf.tensordot(cond,tf.reduce_mean(prior,axis=0),axes=0)
    return cond

In [ ]:
#test the code by calculating the joing probability of the data
r= joint_prob(toy_data,conf_df, head='y')
plt.hist(tf.transpose(r))

In [ ]:
# sample some data and calculate the joint distribution
def one_run_simulation(df,conf_df,dists,sample_no=10000,head='y',features=['x1','x2','x3']):
    #sample all the features
    sim_data = pd.DataFrame(columns=features)
    for feature in features:
        #sample from the feature distribution
        column =  conf_df[conf_df['node']==feature]
        feature_samples = dists[feature].sample(sample_no)
        categories_ = eval(column['encoder'].iloc[0]).categories_[0]
        sim_data[feature] = categories_[np.argmax(feature_samples,axis=1)]
    return joint_prob(sim_data,conf_df, head='y')

# run the simulations muliple times
def repeated_sim(df,conf_df,dists,head='y',features=['x1','x2','x3'],n_samples=10000, repeats=100):
    results=[]
    
    for r in tqdm(range(repeats)):
        result= one_run_simulation(df,conf_df,dists,sample_no=n_samples,head='y',features=features)
        results.append(result)
    return results

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# Baseline output of the model without any modifications to the distributions
dists={'x1':x1_dist,'x2':x2_dist,'x3':x3_dist}

results = repeated_sim(toy_data,conf_df,dists) 

## Experiment 1: Change the distribution of x3

In [ ]:
x3_dist_exp1 = tfd.RelaxedOneHotCategorical(temperature, probs=[.1,.2,.7])
dists={'x1':x1_dist,'x2':x2_dist,'x3':x3_dist_exp1}
results_exp1 = repeated_sim(toy_data,conf_df,dists,n_samples=1000)

In [ ]:
## add functions to facilitate comapriosons between baseline and experiments

def make_cat_dist(x):
    binary_results = np.argmax(x,axis=1)
    probs = np.array([np.sum(binary_results==0), np.sum(binary_results==1)])/binary_results.shape[0]
    return tfd.Categorical(probs=probs)

def kl_divergence(p,q, bins):
    def get_norm_bins (x,bins):
        bin_values= np.histogram(np.vstack(x),bins =bins)[0]
        return bin_values/np.sum(bin_values)
    p_bins = get_norm_bins(p,bins)
    q_bins = get_norm_bins(q,bins)
    result = np.sum(np.where((q_bins!=0) & (p_bins!=0), p_bins*np.log(p_bins/q_bins),0))
    return result,p_bins,q_bins

def compare_results(results_1,results_2,bins=30, color=None):
    plt_results1 = np.vstack(results_1)
    plt_results2 = np.vstack(results_2)
    sns.distplot(plt_results1[:,0],bins=bins,kde=True,norm_hist=True)
    sns.distplot(plt_results2[:,0],bins=bins,kde=True,norm_hist=True,color=color)
    plt.ylim([0,100])
    dist_kl = tfp.distributions.kl_divergence(make_cat_dist(plt_results1),make_cat_dist(plt_results2))
    bin_kl = kl_divergence(results_1,results_2,bins)
    return dist_kl,bin_kl[0]

In [ ]:
print(compare_results(results,results_exp1))
plt.legend(['baseline','exp_1'])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_1.png')

## Experiment 2: Change the distribution of x3 and x1

In [ ]:
x3_dist_exp2 = tfd.RelaxedOneHotCategorical(temperature, probs=[.1,.2,.7])
x1_dist_exp2 = tfd.RelaxedOneHotCategorical(temperature, probs=[.9,.1])
dists={'x1':x1_dist_exp2,'x2':x2_dist,'x3':x3_dist_exp2}
results_exp2 = repeated_sim(toy_data,conf_df,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp2,color='r'))
plt.legend(['baseline','exp_2'])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_2.png')

## Experiment 3: Change the distribution of x2

In [ ]:
x2_dist_exp3 = tfd.RelaxedOneHotCategorical(temperature, probs=[.6,.3,.1])
dists={'x1':x1_dist,'x2':x2_dist_exp3,'x3':x3_dist}
results_exp3 = repeated_sim(toy_data,conf_df,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp3,color='y'))
plt.legend(['baseline','exp_3'])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_3.png')

## Experiment 4: Change the distribution of x1, x2, and x3

In [ ]:
dists={'x1':x1_dist_exp2,'x2':x2_dist_exp3,'x3':x3_dist_exp2}
results_exp4 = repeated_sim(toy_data,conf_df,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp4,color='g'))
plt.legend(['baseline','exp_4'])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_4.png')

## Experiment 5: Change the classifier for m1

In [ ]:
x1_feature = x1_encoder.transform(np.asarray(toy_data['x1']).reshape(-1,1)).toarray()
x2_feature = x2_encoder.transform(np.asarray(toy_data['x2']).reshape(-1,1)).toarray()

feature_names_m1 = np.hstack((x1_encoder.categories_,x2_encoder.categories_))[0]
data_m1 = np.hstack((x1_feature,x2_feature))

feature_columns=[]
for feature_name in feature_names_m1:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_m1 = pd.DataFrame(columns = feature_names_m1,data=data_m1)
train_input_fn = ExpUtils.make_input_fn(data_m1, toy_data['m1_lbl'])
eval_input_fn = ExpUtils.make_input_fn(data_m1, toy_data['m1_lbl'], num_epochs=1, shuffle=False)

#DNN classifier
test_DNN_estimator = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[20, 10],n_classes=3)

test_DNN_estimator.train(train_input_fn)
metrics = test_DNN_estimator.evaluate(input_fn=eval_input_fn)

In [ ]:
print(metrics)

In [ ]:
conf_df_5 = conf_df.copy()
conf_df_5.iloc[3]['estimator'] = 'test_DNN_estimator'
dists={'x1':x1_dist,'x2':x2_dist,'x3':x3_dist}

results_exp5 = repeated_sim(toy_data,conf_df_5,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp5,color='c'))
plt.legend(['baseline','exp_5'])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_5.png')

## Experiment 6: Change the classifier for m2

In [ ]:
# replace a linear classifier with random forest
# build p(x2|f2,f3) 
x2_feature = x2_encoder.transform(np.asarray(toy_data['x2']).reshape(-1,1)).toarray()
x3_feature = x3_encoder.transform(np.asarray(toy_data['x3']).reshape(-1,1)).toarray()

feature_names_m2 = np.hstack((x2_encoder.categories_,x3_encoder.categories_))[0]
data_m2 = np.hstack((x2_feature,x3_feature))

feature_columns=[]
for feature_name in feature_names_m2:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_m2 = pd.DataFrame(columns = feature_names_m2,data=data_m2)


train_input_fn_m2 = ExpUtils.make_input_fn(data_m2, toy_data['m2_lbl'])
eval_input_fn_m2 = ExpUtils.make_input_fn(data_m2, toy_data['m2_lbl'], num_epochs=1, shuffle=False)

tree_est = tf.estimator.BoostedTreesClassifier(feature_columns=feature_columns,
                                               n_batches_per_layer=int(0.5*data_m2.shape[0]/10))
tree_est.train(train_input_fn_m2)
metrics = tree_est.evaluate(eval_input_fn_m2)
print(metrics)

In [ ]:
conf_df_6 = conf_df.copy()
conf_df_6.iloc[4]['estimator'] = 'tree_est'
dists={'x1':x1_dist,'x2':x2_dist,'x3':x3_dist}

results_exp6 = repeated_sim(toy_data,conf_df_6,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp6,color='m'))
plt.legend(['baseline','exp_6'])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_6.png')

## Experiment 7: Replace m1 and m2 with random classifiers

In [ ]:
# Experiment 7
# replace a classifier with a random output

# build p(m2|x2,x3) 
data_m2 = np.hstack((x2_feature,x3_feature))

feature_columns=[]
for feature_name in feature_names_m2:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_m2 = pd.DataFrame(columns = feature_names_m2,data=data_m2)

random_lbl = toy_data['m2_lbl'][np.random.permutation(toy_data.shape[0])]
train_input_fn_m2 = ExpUtils.make_input_fn(data_m2, random_lbl)
eval_input_fn_m2 = ExpUtils.make_input_fn(data_m2, random_lbl, num_epochs=1, shuffle=False)

random_est = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=2)
random_est.train(train_input_fn_m2)
result = random_est.evaluate(eval_input_fn_m2)

In [ ]:
conf_df_7a = conf_df.copy()
conf_df_7a.iloc[4]['estimator'] = 'random_est'
dists={'x1':x1_dist,'x2':x2_dist,'x3':x3_dist}

results_exp7a = repeated_sim(toy_data,conf_df_7a,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp7a,color='#FFA500'))
plt.legend(['baseline','Randomly trained $m_2$'])
plt.xlim([0.39,0.55])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_7a.png')

In [ ]:
# Experiment 7b
# replace the other classifier with a random output

# build p(m1|x1,x2) 
feature_names_m1 = np.hstack((x1_encoder.categories_,x2_encoder.categories_))[0]
data_m1 = np.hstack((x1_feature,x2_feature))

feature_columns=[]
for feature_name in feature_names_m1:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))
data_m1 = pd.DataFrame(columns = feature_names_m1,data=data_m1)

random_lbl = toy_data['m2_lbl'][np.random.permutation(toy_data.shape[0])]
train_input_fn = ExpUtils.make_input_fn(data_m1, random_lbl)
eval_input_fn = ExpUtils.make_input_fn(data_m1, random_lbl, num_epochs=1, shuffle=False)

random_est_m1 = tf.estimator.LinearClassifier(feature_columns=feature_columns,n_classes=3)
random_est_m1.train(train_input_fn)
result = random_est_m1.evaluate(eval_input_fn)

In [ ]:
conf_df_7b = conf_df.copy()
conf_df_7b.iloc[3]['estimator'] = 'random_est_m1'
dists={'x1':x1_dist,'x2':x2_dist,'x3':x3_dist}

results_exp7b = repeated_sim(toy_data,conf_df_7b,dists,n_samples=1000)

In [ ]:
print(compare_results(results,results_exp7b,color='#FF6347'))
plt.legend(['baseline','Randomly trained $m_1$'])
plt.xlim([0.39,0.55])
plt.xlabel('Solution Output Probability')
plt.ylabel('Density')
plt.savefig('exp_7b.png')